# Лабораторна робота 3 з Теоретико-числових алгоритмів в криптології
## Тема: Реалiзацiя та застосування алгоритму дискретного логарифмування **index-calculus**

**Варіант:** 2\
**Виконали:** Бондар Петро, Кістаєв Матвій\
**Група:** ФІ-03

### Посилання
Github: https://github.com/Pechenkya/NTA-Labs-Bondar-Kistaiev-FI-03/tree/main/Lab-3

Docker image: https://hub.docker.com/r/petrob2003/nta_lab-3_bondar_kistaiev

In [9]:
import numpy as np
import time
from math import exp, sqrt, log, gcd
# Use prime checking from Lab 1
from factor_module import check_prime

## Генеруємо факторну базу

Обмеження використовується рекомендоване в методичних матеріалах:
\begin{equation}
B = c\cdot e^{\frac{1}{2}(\log{n} \cdot \log{\log{n}})^{\frac{1}{2}}},\quad c = 3.38
\end{equation}

Для перевірки чисел на простоту використовується алгоритм Міллера-Рабіна.

In [10]:
lb = 2

def gen_factor_base(n):
    c = 3.38 # recommended constant
    B_lim = int(c * exp(0.5 * sqrt(log(n, lb) * log(log(n, lb), lb))))
    print(f"Factor Base limit: {B_lim}")
    
    S = []
    for a in range(2, B_lim):
        if check_prime(a):
            S.append(a)
    
    return S

## Генеруємо систему порівнянь 

Кількість визначається наступним чином: $|S| + C$, де $S$ - факторна база, а $C = 20$ - підібрана константа.

Гладкість і розклад по факторній базі визначаються за допомогою пробних ділень по факторній базі (ефективніше, ніж виконувати факторизацію).

### Прямий підхід (без розпаралелювання)

Послідовно підносимо $\alpha$ до степенів, для кожного степеня перевіряємо отримане число на гладкість. Послідовне піднесення в цьому випадку значно пришвидшує алгоритм, порівняно з випадковим вибором степеня і пошуку розкладу конкретно під нього.

In [11]:
C = 20

def is_smooth(val, S):
    D = [0] * len(S)
    for i in range(len(S)):
        while val % S[i] == 0:
            D[i] += 1
            val //= S[i]
        
    if val != 1:
        return [False, []]
    else:
        return [True, D]


def gen_equations(a, n, S, number_of_eq=0):
    st = time.time()
    if number_of_eq == 0:
        number_of_eq = len(S) + C

    A = []
    b = []

    curr_power = 1
    curr_val = a

    while len(b) < number_of_eq:
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth:
            A.append(pows)
            b.append(curr_power)

        curr_val = (curr_val * a) % n
        curr_power += 1

        if curr_val == 1:
            break
    et = time.time()

    print(f"Generated equations: {len(b)}")
    print(f"Generating time: {et - st} seconds\n")
    return (np.array(A), np.array(b))

### Підхід з розбиттям на підзадачі (з розпаралелюванням)

Розбиваємо задачу на 4 підзадачі, на кожній запускаємо послідовне піднесення $\alpha$ до степенів, починаючи з якогось конкретного степеня. Отримані розклади збираються в єдиний масив.

Конкретно в даній реалізації, через певні умовності мови, така реалізація виявляється повільнішою навіть для великих значеннь модуля.

In [12]:
from multiprocessing import Process, Manager
Subprocesses = 4

def equation_subprocess(a, n, S, start_power, A_shared, b_shared, number_of_eq):
    print("Started subprocess")
    curr_power = start_power
    curr_val = pow(a, start_power, n)

    while len(b_shared) < number_of_eq:
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth:
            A_shared.append(pows)
            b_shared.append(curr_power)

        curr_val = (curr_val * a) % n
        curr_power += 1

        if curr_val == 1:
            break

def gen_equations_parallel(a, n, S, number_of_eq=0):
    st = time.time()
    if number_of_eq == 0:
        number_of_eq = len(S) + C

    with Manager() as manager:
        A_shared = manager.list()
        b_shared = manager.list()

        pow_step = (n-1) // Subprocesses
        pow_start = 1
        p_handles = []
        # Start processes
        for _ in range(Subprocesses):
            p_handles.append(Process(target=equation_subprocess, args=(a, n, S, pow_start, A_shared, b_shared, number_of_eq)))
            p_handles[-1].run()
            p_handles[-1].start()
            pow_start += pow_step

        for i in range(Subprocesses):
            p_handles[i].join()
        et = time.time()

        print(f"Generated equations: {len(b_shared)}")
        print(f"Generating time: {et - st} seconds\n")
        return (np.array(A_shared), np.array(b_shared))

## Розв'язування системи лінійних порівнянь за модулем

Для розв'язання цієї задачі ми використали "підкоректований" алгорим Гаусса. Так як оберені за складеним модулем існують не для всіх елементів, опорними елементами обираються тільки елементи, які ми можемо обернути (або обернути після скорочення всього рядка на НСД модуля та цього елемента). Всі інші операції залишаються тими самими.

Таким чином ми ефективно здатні визначити точний розв'язок системи лінійних порівнянь.

In [13]:
def solve_modular_eq(A_in, b, mod):
    m = len(A_in[0])
    n = len(A_in)
    A = np.concatenate([A_in, np.transpose([b])], axis=1, dtype='object')

    chosen = []

    # Gauss elimination
    for j in range(m):
        found = False
        for i in range(n):
            if chosen.count(i) != 0:
                continue

            if gcd(A[i][j], mod) == 1:
                chosen.append(i)
                found = True

                inv = pow(A[i][j], -1, mod)
                A[i] = A[i] * inv % mod
                
                for k in range(n):
                    if k != i and A[k][j] != 0:
                        A[k] = (A[k] - A[k][j]*A[i]) % mod
                
                break
        
        if not found:
            for i in range(n):
                if chosen.count(i) != 0 or A[i][j] == 0:
                    continue

                d = gcd(A[i][j], mod)
                if np.all(A[i] % d == 0):
                    chosen.append(i)
                    A[i] //= d

                    inv = pow(A[i][j], -1, mod)
                    A[i] = A[i] * inv % mod
                    for k in range(n):
                        if k != i and A[k][j] != 0:
                            A[k] = (A[k] - A[k][j]*A[i]) % mod
                    break

    # print(A)
    solution = []
    for j in range(m):
        added = False
        for i in range(n):
            if A[i][j] != 0:
                solution.append(A[i][-1])
                added = True
                break
        if not added:
            solution.append(0)
    


    return np.array(solution)

## Знаходимо відповідний $\log_\alpha \beta$

Складено два алгоритми:
1. Не використовує розв'язування системи порівнянь за модулем, а перебором шукає розклад $\beta\cdot\alpha^l$, який відповідаю деякому розкладу $\alpha^k$.
2. Ефективний алгоритм, який використовує розв'язок системи, щоб визначити значення $\log{(\beta\cdot\alpha^l)}$.

Обидва алгоритмів повертають розв'язок відніманням $k - l$, якщо такий розв'язок існує, інакше - повертають помилку. Через свою природу, перший алгоритм працює значно повільніше за другий.

In [14]:
def find_index_sequential_comparing(a, beta, n, S, A, b):
    curr_ind = 0
    curr_val = beta

    A_list = A.tolist()
    for _ in range(n-1):
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth and A_list.count(pows) != 0:
            corr_a_ind = b[A_list.index(pows)]
            return (corr_a_ind - curr_ind) % (n - 1)
        
        curr_ind += 1
        curr_val = (curr_val * a) % n

    raise RuntimeError("Can't find index!")

def find_index(a, beta, n, S, S_idxs):
    curr_ind = 0
    curr_val = beta

    for _ in range(n-1):
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth:
            corr_a_ind = np.dot(S_idxs, pows)
            return (corr_a_ind - curr_ind) % (n - 1)
        
        curr_ind += 1
        curr_val = (curr_val * a) % n

    raise RuntimeError("Can't find index!")

## Загальний алгоритм розв'язання

Алгоритм складається з трьох кроків:
1. Згенерувати факторну базу $S$.
2. Згенерувати систему лінійних порівнянь по заданим параметрам.
3. (для основного алгоритму) Розв'язати систему порівнянь.
4. Знайти логарифм за допомогою відповідного підходу.

In [15]:
def solve_brute(alpha, beta, n):
    Base = gen_factor_base(n)
    A, b = gen_equations(alpha, n, Base)
    res = find_index_sequential_comparing(alpha, beta, n, Base, A, b)
    return res

def solve(alpha, beta, n):
    Base = gen_factor_base(n)
    A, b = gen_equations(alpha, n, Base)
    S_idxs = solve_modular_eq(A, b, n - 1)
    res = find_index(alpha, beta, n, Base, S_idxs)
    return res

## Підсумок

### Приклад розв'язання задачі на конкретному прикладі, згенерованому за допомогою допоміжної програми

Довжина модуля в десятковому представленні: 15

**Вхідні дані:**\
a = 594817592195997\
b = 241235639117967\
p = 686825437261057

In [16]:
st = time.time()
res = solve(594817592195997, 241235639117967, 686825437261057)
et = time.time()

print(f"\nGeneral execution time: {et - st} seconds\n")
print(f"Result: {res}")

Factor Base limit: 13929
Generated equations: 1667
Generating time: 13.641370296478271 seconds

[[1 0 0 ... 0 0 357297226398900]
 [0 0 0 ... 0 0 459715542945663]
 [0 1 0 ... 0 0 75816976740998]
 ...
 [0 0 0 ... 0 0 638742465430213]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
General execution time: 96.61787414550781 seconds

Result: 242042764040700


### Заміри часу виконання і порівняння з рішенням з другої лабораторної роботи

$$ \begin{array}{|c|c|r|r|r|r|c|c|c|}
    \hline № & \text{Test Level} & \alpha & \beta & \mod & \text{Solution} & \text{SPH time (sec)} & \text{Index-Calculus time (sec)} & \text{Bruteforce (sec)}\\
    \hline 1 & 3\;(1) & 405 & 162 & 863 & 602 & 0.0046                      & 0.0040 & 0.0000\\
    \hline 2 & 3\;(2) & 91 & 54 & 983 & 490 &  0.0045                        & 0.0036& 0.0000\\
    \hline 3 & 4\;(1) & 1735 & 3267 & 5443 & 2877   & 0.0090                 & 0.0062& 0.0000\\
    \hline 4 & 4\;(2) & 5606 & 766 & 5807 & 5787 & 0.0338                    & 0.0050& 0.0000\\
    \hline 5 & 5\;(1) & 53100 & 16911 & 64567 & 47387   & 0.0020             & 0.0159& 0.0030\\
    \hline 6 & 5\;(2) & 236 & 21489 & 32069 & 25264 & 0.0730                 & 0.0095& 0.0020\\
    \hline 7 & 6\;(1) & 324217 & 96569 & 397519 & 292529 & 0.0025            & 0.0251& 0.0217\\
    \hline 8 & 6\;(2) & 341681 & 719645 & 933397 & 360413 & 0.7903           & 0.0306& 0.0266\\
    \hline 9 & 7\;(1) & 416859 & 811893 & 1100641 & 849195 & 0.0216          & 0.0491& 0.0600\\
    \hline 10 & 7\;(2) & 698063 & 1842530 & 4948849 & 4639227 & 0.3340       & 0.0502& 0.3651\\
    \hline 11 & 8\;(1) & 10329729 & 5996667 & 12879569 & 3681443 & 0.0319    & 0.0643& 0.2640\\
    \hline 12 & 8\;(2) & 40339922 & 8665466 & 46835927 & 36750957 & 1.6040   & 0.1106& 2.6295\\
    \hline 13 & 9\;(1) & 133831429 & 394869515 & 603047449 & 575130035 & 0.0110 & 0.3050& 42.387\\
    \hline 14 & 9\;(2) & 105885086 & 242684147 & 281243987 & 116787747 & 0.1408 & 0.2262& 8.6493\\
    \hline 15 & 10\;(1) & 3254326800 & 5738480278 & 6821486569 & 5425105225 & 0.1868                        & 0.8527& -\\
    \hline 16 & 10\;(2) & 2503970251 & 4616555134 & 7687535143 & 3612269744 & 0.7432                        & 0.8898&-\\
    \hline 17 & 11\;(1) & 15755281373 & 2216307038 & 27834916511 & 3740804930 & 0.0680                      & 1.3528&-\\
    \hline 18 & 11\;(2) & 22853910777 & 25817503193 & 28791230497 & 21096709669 &  -                         & 1.4848&-\\
    \hline 19 & 12\;(1) & 507709796487 & 765316699585 & 808811929619 & 454007131294 & 0.0195                     & 5.8950&-\\
    \hline 20 & 12\;(2) & 103462240942 & 347520779949 & 423107943311 & 104026002582 & 0.8080                     & 4.9429&-\\
    \hline 21 & 13\;(1) & 575769640533 & 1587103349320 & 2679503891797 & 2508924257003 & 0.2689                  & 11.157&-\\
    \hline 22 & 13\;(2) & 5193719065806 & 433642518389 & 5200738710043 & 3299582449828 & -                  & 12.986&-\\
    \hline 23 & 14\;(1) & 69729143465395 & 51717474377131 & 81827408544181 & 70597713698775 & 0.1735             & 36.710&-\\
    \hline 24 & 14\;(2) & 9342677045103 & 5678740808699 & 34191150382817 & 26463769397737 & -               & 26.645&-\\
    \hline 25 & 15\;(1) & 58035538961226 & 84400445924237 & 124360974085129 & 13651448394237 & 0.2443            & 48.763&-\\
    \hline 26 & 15\;(2) & 137042521277099 & 481703362714446 & 648235951112423 & 266893171218140 & -         & 85.634&-\\
    \hline 27 & 16\;(1) & 1646807569044576 & 1357634646373857 & 2292878607986357 & 547317542238434 & 0.0864      & 152.45&-\\
    \hline 28 & 16\;(2) & 783529233531527 & 1339896166988741 & 1486639524290603 & 318864505892145 & -       & 129.01&-\\
    \hline
\end{array}$$

Числа довжини 17 і більше дуууже довго генеруються допоміжною програмою :)

#### Графіки (Кількість цифр - Час роботи)

![pic1](../Images/small.png "Title")

![pic1](../Images/big.png "Title")